# Confidence Intervals and Output Analysis
This shows how to compute/interpret confidence intervals of quantities of interest based on our sampled values.

Recall that for large enough $N$, using the sample mean, $\bar{Y}$, and sample standard deviation, $S$, the $(1-\alpha)\times 100$ % confidence interval is:
$$
\bar{Y} \pm z_{1-\alpha/2}\frac{S}{\sqrt{N}}
$$

In [ ]:
using Plots
using Random
using Distributions
using LinearAlgebra
using Printf
using HypothesisTests

In [ ]:
# for readability, these are good settings to use
default(xtickfontsize=14,  ytickfontsize=14, ztickfontsize=14,
    guidefontsize=14, legendfontsize=12, lw=2,ms=8)

# Example
In the first example, we try to estimate the mean of an exponential random variable with parameter $\lambda>0$.  Recall that the mean is $\lambda$.  This is a highly non-Gaussian distribution.

In [ ]:
λ_true = 1;
exp_dist = Exponential(λ_true);



The $1-\alpha/2$ quantile, $z_{1-\alpha/2}$, associated $\Phi$, the $N(0,1)$ CDF can be obtained using `Distributions` with the following code:

In [ ]:
α = 0.05;
z=quantile(Normal(), 1-α/2);
@show z;

Sample and compute the associated CI

In [ ]:
N = 10^1;

Random.seed!(100)
samples = rand(exp_dist, N);
CI = (mean(samples) - std(samples)*z/sqrt(N), 
    mean(samples) + std(samples)*z/sqrt(N));
@show CI;

Alternatively, compute this using the `HypothesisTests.jl` package:

In [ ]:
Ztest = OneSampleZTest(mean(samples), std(samples), N);
@show Ztest;

Verify that approximately $(1-\alpha)\times 100$% of all numerical CIs contain the true value;

In [ ]:
n_indep_trials = 10^4;
sample_means = zeros(n_indep_trials);

n_contain = 0; # count the number of CIs that contain the true value

Random.seed!(100)
for j in 1:n_indep_trials
    samples = rand(exp_dist, N);
    sample_means[j] = mean(samples);
    CI = (mean(samples) - std(samples)*z/sqrt(N), 
        mean(samples) + std(samples)*z/sqrt(N));
    if (λ_true< CI[2]) && (λ_true> CI[1])
        n_contain+=1;
    end
end
@show n_contain/n_indep_trials;


Verify the CLT, that, as $N$ gets large, the sample means are Gaussian.

In [ ]:
histogram(sample_means,label="Sample Means")
xlabel!("x")
ylabel!("Frequency")
title!(@sprintf("Sample Mean Distribution with N = %d", N))